In [145]:
# Build image
!docker build . --quiet --tag code-scanner:latest

sha256:4f8924421361545261baa528e2e145615194605b1d47ac943b1b72856b4ec895


In [146]:
# Run container
!docker run -d -p 8080:8080 --name test-code-scanner code-scanner:latest --host 0.0.0.0 --port 8080

6d8b24f40bbee5bee337d031e536f3b4843bb48b1954d60e5afadf3a045e8159


In [147]:
# check container logs
!docker logs test-code-scanner

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


In [144]:
# kill and remove container
!docker kill test-code-scanner && docker rm test-code-scanner

test-code-scanner
test-code-scanner


In [140]:
# list contexts to get docker host DNS
!docker context ls

NAME                              DESCRIPTION                               DOCKER ENDPOINT                                              ERROR
default                           Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                                  
m5.xlarge_i-0acf416220d9e58d1 *                                             tcp://ip-172-31-72-55.ap-southeast-2.compute.internal:1111   


In [78]:
# IP from docker context when using Studio Docker CLI extension
host_ip = "ip-172-31-72-55.ap-southeast-2.compute.internal"

In [148]:
%%bash -s $host_ip
curl ${1}:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   3623      0 --:--:-- --:--:-- --:--:--  4400


{"Health_Check":"200"}

In [149]:
%%bash -s $host_ip
curl -X POST ${1}:8080/scan -d '{"script": "from flask import Flask\napp = Flask(__name__)\nif __name__ == \"__main__\":\n    app.run(debug=True)", "language": "Python"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   427  100   290  100   137     55     26  0:00:05  0:00:05 --:--:--    95


[{"Start line":4,"End line":4,"Recommendations":"Detected Flask app with debug=True. Do not deploy to production with this flag enabled as it will leak sensitive information. Instead, consider using Flask configuration variables or setting 'debug' using system environment variables.\n\n"}]

In [150]:
%%bash -s $host_ip
curl -X POST ${1}:8080/scan -d '{"script": "import os\nuser_id = input(\"enter some text\")\nos.system(f\"echo {user_id}\")", "language": "Python"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   117    0     2  100   115      0     22  0:00:05  0:00:05 --:--:--     0


[]

In [133]:
import os
user_id = input("enter some text")
os.system(f"echo {user_id}")

enter some text Hello


Hello


0

In [2]:
!docker pull returntocorp/semgrep

Using default tag: latest
latest: Pulling from returntocorp/semgrep

a8db6415: Pulling fs layer 
d5e70e42: Pulling fs layer 
a579cd50: Pulling fs layer 
8c10ee34: Pulling fs layer 
b3fa61f8: Pulling fs layer 
3b615cde: Pulling fs layer 
59e5b97d: Pulling fs layer 
583c3bde: Pulling fs layer 
bdb8814b: Pulling fs layer 
596a008a: Pulling fs layer 
f0b32f57: Pulling fs layer 
0a81459b: Pulling fs layer 
27999d47: Pulling fs layer 
b8ca7cf4: Pulling fs layer 
46bf2140: Pulling fs layer 
40b4f99a: Pull complete  200B/200B7kBBDigest: sha256:b4637a27abf1e49aeab753fdfd41d917493d8fc7f6da9de2daf109b4d7369fe8
Status: Downloaded newer image for returntocorp/semgrep:latest
docker.io/returntocorp/semgrep:latest


In [48]:
!docker run -v ./:/opt/ml/code/ returntocorp/semgrep which semgrep

/usr/local/bin/semgrep


In [143]:
!docker run -v ./:/opt/ml/code/ returntocorp/semgrep semgrep scan --registry-caching -q --json --disable-version-check --config=auto --output=/opt/ml/code/scan.json /opt/ml/code/script.py

Error: fatal error
Exiting with error status 2: osemgrep scan --registry-caching -q --json --disable-version-check --config=auto --output=/opt/ml/code/scan.json /opt/ml/code/script.py


In [91]:
import json

with open("scan.json", "r") as json_file:
    scan_results = json.load(json_file)

scan_results.keys()

dict_keys(['errors', 'paths', 'results', 'skipped_rules', 'version'])

In [101]:
output = {}
for result in scan_results["results"]:
    line_tuple = f'{result["start"]["line"]}:{result["end"]["line"]}'
    if line_tuple not in output:
        output[line_tuple] = ""
    output[line_tuple] += f'{result["extra"]["message"]}\n\n'
results = []
for out in output:
    results.append({"Start": int(out.split(":")[0]), "End": int(out.split(":")[1]), "Recommendations": output[out]})
    
print(results)

[{'Start': 10, 'End': 13, 'Recommendations': "Request data detected in os.system. This could be vulnerable to a command injection and should be avoided. If this must be done, use the 'subprocess' module instead and pass the arguments as a list. See https://owasp.org/www-community/attacks/Command_Injection for more information.\n\n"}, {'Start': 10, 'End': 18, 'Recommendations': "User-controlled data from a request is passed to 'execute()'. This could lead to a SQL injection and therefore protected information could be leaked. Instead, use django's QuerySets, which are built with query parameterization and therefore not vulnerable to sql injection. For example, you could use `Entry.objects.filter(date=2006)`.\n\n"}, {'Start': 13, 'End': 13, 'Recommendations': "User data detected in os.system. This could be vulnerable to a command injection and should be avoided. If this must be done, use the 'subprocess' module instead and pass the arguments as a list.\n\nFound user-controlled data used 